<a href="https://colab.research.google.com/github/OCCI-Staar/Curve-Maps-With-Clouds/blob/main/Curve_Maps_SHP_Clouds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note this will **NOT** filter out clouds
## Create a standard environment

*   Simply Click play button to run all scripts

*   If no errors, continue to 1)

In [2]:
#Just need the required versions from GitHub
!wget -O required-info.txt https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-With-Clouds/main/required-info.txt

--2024-06-20 19:36:34--  https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-With-Clouds/main/required-info.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129 [text/plain]
Saving to: ‘required-info.txt’

required-info.txt   100%[===================>]     129  --.-KB/s    in 0s      

2024-06-20 19:36:34 (2.12 MB/s) - ‘required-info.txt’ saved [129/129]



In [3]:
# View the content of required-info.txt
!cat required-info.txt

# Install packages listed in required-info.txt
!pip install -r required-info.txt

pandas==2.2.2 
geopandas==0.14.4 
shapely==2.0.2 
fiona==1.9.5 
pyproj==3.6.1 
geetools==0.6.11 
earthengine-api==0.1.272
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.2/229.2 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for geetools: filename=geetools-0.6.11-py3-none-any.whl size=90202 sha256=ef15c23017c30889979d44bf4576f3434679967a07e1a7e54186cf31264c657d
  Stored in directory: /root/.cache/

In [ ]:
#Needed Libraries
import ee
import time
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from datetime import datetime
from geetools import batch
import ipywidgets as widgets

from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'geetools'

In [ ]:
#Build Docker
!docker build -t maps-docker-image .


# 1) Authenticate yourself through Google Earth Engine

In [ ]:
#Authenticate the GEE User
ee.Authenticate()
# Verify if GEE server was authenticated
try:
  ee.Initialize()
except ee.EEException as error:
  print(f'ERROR: {error}')

NameError: name 'ee' is not defined

# 2) Supply Satellite, Date Range, Field Name

In [ ]:
satellite = "COPERNICUS/S2_SR_HARMONIZED" # @param ["LANDSAT/LT04/C01/T1_SR", "LANDSAT/LT05/C01/T1_SR", "LANDSAT/LE07/C01/T1_SR", "LANDSAT/LC08/C01/T1_SR", "COPERNICUS/S2_SR_HARMONIZED"]
first_date = '2023-06-09'  #@param {type: "date"}
last_date  = '2023-10-03'  #@param {type: "date"}
field_name = "L1" #@param {type:"string"}


if satellite == "COPERNICUS/S2_SR_HARMONIZED":
      sat = 'S2SR'
if satellite == "LANDSAT/LT04/C01/T1_SR":
      sat = 'LT04'
if satellite == "LANDSAT/LT05/C01/T1_SR":
      sat = 'LT05'
if satellite == "LANDSAT/LE07/C01/T1_SR":
      sat = 'LE07'
if satellite == "LANDSAT/LC08/C01/T1_SR":
      sat = 'LC08'

# 3) Choose Field Boundaries file you want processed.


*   File extension must be **.shp**


In [ ]:
from google.colab import files
shp_file = files.upload()

# 4) Time to process and export images

In [ ]:
#With Clouds
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import satellite info
    s2_sr_col = (ee.ImageCollection(satellite)
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    return s2_sr_col

In [ ]:
#Pulls geometry from the polygon
def get_bbox_coords(pol):
    shp_bb = shp_file[:-5] + '_bb.geojson'
    #Convert polygon CRS to WGS84 DD
    pol = pol.to_crs({'init': 'epsg:4326'})
    #Create a buffer for the boundary and save as geoJSON
    pol.envelope.to_file(filename=shp_bb, driver="GeoJSON")
    #Open GeoJSON
    geom = json.loads(pol.envelope.to_json())['features'][0]['geometry']

    return(geom)

In [ ]:
pol = gpd.read_file(shp_file)

pol = pol.dissolve()

pol = pol.buffer(0.005)

geom = get_bbox_coords(pol)

#Get coordinates from GeoJSON
coords = geom['coordinates']
regions = ee.Geometry.Polygon(coords)

start_date = ee.Date(first_date)
end_date = ee.Date(last_date)

scale = 10
extra = dict(sat=sat)
s2_sr_cld_col = get_s2_sr_cld_col(regions, start_date, end_date)
imagery = s2_sr_cld_col
field_name_sat = f'{field_name}'+"{id}"

print(imagery.size().getInfo())
batch.Export.imagecollection.toDrive(imagery, folder=f'YieldCurve_{sat}_{field_name}',
                                     region=regions, scale=scale, verbose = True,
                                     namePattern = field_name_sat)
time.sleep(300)
print("Export tasks were a success!")